In [1]:
import os
import librosa
import sys , re
import unicodedata
import os.path as op

from pathlib import Path
from scipy.io import wavfile
import pandas as pd
import pyarabic.araby as araby

import numpy as np
import wave
import soundfile as sf

from tqdm import tqdm
from datasets import load_from_disk

In [2]:
df = load_from_disk('/l/users/speech_lab/QASR_TTS/QASRTTS_HF')
df

DatasetDict({
    train: Dataset({
        features: ['audio_path', 'arabic_text', 'buckwalter_text', 'audio'],
        num_rows: 2236
    })
    validation: Dataset({
        features: ['audio_path', 'arabic_text', 'buckwalter_text', 'audio'],
        num_rows: 195
    })
})

In [6]:
df['validation'][77]['audio_path'], df['validation'][77]['arabic_text']

('/l/users/speech_lab/QASR_TTS/Khadija_wav/wavs/432F62C3-D353-463B-8138-98733286F1D4_48.wav',
 '')

In [3]:
root  = '/l/users/speech_lab/QASR_TTS'
Path(df['train'][0]['audio_path']).relative_to(root)

PosixPath('Khadija_wav/wavs/1D195228-25FD-4006-8702-AA2059345B26_59.wav')

In [4]:
map_numbers = {'0': '٠', '1': '١', '2': '٢', '3': '٣', '4': '٤', '5': '٥', '6': '٦', '7': '٧', '8': '٨', '9': '٩'}
map_numbers = dict((v, k) for k, v in map_numbers.items())
punctuations = ''.join([chr(i) for i in list(i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P'))])
punctuations = punctuations + '÷#ݣ+=|$×⁄<>`åûݘ ڢ̇ پ'
def remove_punctuation(word):
    return word.translate(str.maketrans('', '', re.sub('[@% ]','', punctuations))).lower()

def remove_diacritics(word):
    return araby.strip_diacritics(word)

def convert_numerals_to_digit(word):
    sentence=[]
    for w in word:
        sentence.append(map_numbers.get(w, w))
    word = ''.join(sentence)
    return word

def preprocess_arabic_text_v1(text):
    # text = preprocess_arabic_twitter_text(text)
    text = remove_diacritics(text)
    text = convert_numerals_to_digit(text)
    text = remove_punctuation(text)
    return text

In [22]:
df['train'].shape[0]

(2236, 4)

In [46]:
sf.write(df['validation'][128]['audio_path'],df['validation'][128]['audio'],22050 )

In [5]:
# splits = {'train':'train', 'validation':'valid'}
splits = {'validation':'valid'}
female = "spk_emb/qasr_KhadijaBenguenna_embedding.npy"
male = "spk_emb/qasr_MahmoudMourad_embedding.npy"

for key, split in splits.items():
    save_tsv = f"/l/users/speech_lab/_SpeechT5PretrainDataset/Finetune/TTS/hubert_labels/QASRTTS/{split}.tsv"
    save_txt = f"/l/users/speech_lab/_SpeechT5PretrainDataset/Finetune/TTS/labels/QASRTTS/{split}.txt"
    tsv = open(save_tsv, 'w')
    txt = open(save_txt, 'w')
    print(f"{root}", file=tsv)
    for row in tqdm(df[key], total=df[key].shape[0], desc=f"Processing {split} split"):
        # try:
            # assert Path(row['audio_path']).exists(), f"{row['audio_path']} does not exist"
            # print(f"audio: {row['audio_path']}")
            # wav,sr = librosa.load(row['audio_path'])
            # wav = librosa.resample(wav, sr, 16000)
            # wavfile.write(row['audio_path'], 16000, wav)
        if row['audio_path'] == "Khadija_wav/wavs/432F62C3-D353-463B-8138-98733286F1D4_48.wav":
            continue
        else:
            sr, wav = wavfile.read(row['audio_path'])
            assert sr == 16000, f"sampling rate {sr} != 16000"
            n_frames = wav.shape[0]
            audio_path = Path(row['audio_path']).relative_to(root)
            spk_emb = female if Path(row['audio_path']).parts[5] == 'Khadija_wav' else male
            print(f"{audio_path}\t{n_frames}\t{spk_emb}", file=tsv)
            print(f"{preprocess_arabic_text_v1(row['arabic_text'])}", file=txt)
        # except AssertionError:
        #     # wavfile.write(np.array(row['audio_path']), 22050, row['audio'])
        #     # wav,sr = librosa.load(row['audio_path'])
        #     print(f"Resampling {row['audio_path']}")
        #     wav = librosa.resample(np.array(row['audio']), 22050, 16000)
        #     sf.write(row['audio_path'],wav, sr=16000)
        #     wav = wave.open(row['audio_path'], 'r')
        #     n_frames = wav.getnframes()
        #     # sr, wav = wavfile.read(row['audio_path'])
        #     # n_frames = wav.shape[0]
        #     audio_path = Path(row['audio_path']).relative_to(root)
        #     spk_emb = female if Path(row['audio_path']).parts[5] == 'Khadija_wav' else male
        #     print(f"{audio_path}\t{n_frames}\t{spk_emb}", file=tsv)
        #     print(f"{remove_punctuation(row['arabic_text'])}", file=txt)
    tsv.close()
    txt.close()




Processing valid split:   0%|          | 0/195 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/l/users/speech_lab/QASR_TTS/Khadija_wav/wavs/7DB7600C-D931-49BE-90FA-DE5C241B18D8_61.wav'